## Data Analysis

In [1]:
! pip install transformers evaluate datasets requests pandas sklearn nltk

# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import torch

from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=Tr

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [3]:
# Read the data from directory
train_data = pd.read_csv("/kaggle/input/techno/train.csv")
test_data = pd.read_csv("/kaggle/input/techno/test.csv")
test_labels = pd.read_csv("/kaggle/input/techno/test_labels.csv")
#sample_submission = pd.read_csv(working_directory + "sample_submission.csv")

In [4]:
# Checking sample submission to get an idea
#print(sample_submission.head())

In [5]:
test_labels.head()

,id,toxic
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0


In [6]:
train_data = train_data.drop(columns=['id'], axis=1)
train_data.head()

,feedback_text,toxic,abusive,vulgar,menace,offense,bigotry
0,:::Sounds good. Let me know when you're done ...,0,0,0,0,0,0
1,"""\nI say something, but it didn't actually con...",0,0,0,0,0,0
2,"""Agustina Barrientos]] \n | Modelo de Piñeiro ...",0,0,0,0,0,0
3,FYI I enjoy licking strangers scrotal sacks......,1,0,1,0,0,0
4,How do you get a site?\nMany penguins have ask...,0,0,0,0,0,0


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23473 entries, 0 to 23472
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   feedback_text  23473 non-null  object
 1   toxic          23473 non-null  int64 
 2   abusive        23473 non-null  int64 
 3   vulgar         23473 non-null  int64 
 4   menace         23473 non-null  int64 
 5   offense        23473 non-null  int64 
 6   bigotry        23473 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [8]:
train_data.columns

Index(['feedback_text', 'toxic', 'abusive', 'vulgar', 'menace', 'offense',
       'bigotry'],
      dtype='object')

In [9]:
train_data.columns = train_data.columns.str.strip()
train_data.describe()

,toxic,abusive,vulgar,menace,offense,bigotry
count,23473.000000,23473.000000,23473.000000,23473.000000,23473.000000,23473.000000
mean,0.093554,0.007498,0.054318,0.002812,0.048992,0.008222
std,0.291214,0.086267,0.226648,0.052952,0.215857,0.090305
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
train_data.isnull().sum()

feedback_text    0
toxic            0
abusive          0
vulgar           0
menace           0
offense          0
bigotry          0
dtype: int64

In [11]:
labels = list(train_data.columns.values)
labels = labels[2:]
print(labels)

['abusive', 'vulgar', 'menace', 'offense', 'bigotry']


In [12]:
counts = []
for label in labels:
    counts.append((label, train_data[label].sum()))
df_stats = pd.DataFrame(counts, columns=['label', 'number_of_comments'])
df_stats

,label,number_of_comments
0,abusive,176
1,vulgar,1275
2,menace,66
3,offense,1150
4,bigotry,193


In [13]:
total_labeled_sample = sum(df_stats.number_of_comments)
total_labeled_sample

2860

In [14]:
test_data.head()


,id,content,lang
0,56496,"Bahar merhaba, Şu TV programı için bildiğin gi...",tr
1,18266,"Merhaba, başlık taşıma nedeniyle bir karışıklı...",tr
2,37852,kullanıcı ne mutlu türküm diyene yazmış ta...,tr
3,25075,Quand la plaisanterie sur le synopsis de ce fi...,fr
4,330,Não fosse seu esclarecimento no blog da Anne (...,pt


## Preprocessing the data

In [15]:
import re
def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'[0-9]+' , '' ,text)
    text = re.sub(r'\s([@][\w_-]+)', '', text).strip()
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("#" , " ")
    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string

### Data Splitting

In [16]:
# split input and output
#X = df['feedback_text']
#y = df.drop(columns=['feedback_text'], axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
X = train_data['feedback_text']
Y = train_data.drop(columns=['feedback_text'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

### Tokenizing the Data

In [18]:
from transformers import BertTokenizer

#Load the Bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

# Create a funcition to tokenize a set of text

def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []
    
    
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text = text_preprocessing(sent), #preprocess sentence
            add_special_tokens = True,
            max_length= MAX_LEN  ,
            pad_to_max_length = True,     
            return_attention_mask= True
        )
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
        
    #convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    return input_ids,attention_masks

In [19]:
MAX_LEN = 150
# Get input ids and attention masks after preprocessing the text
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
test_inputs, test_masks = preprocessing_for_bert(X_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Tokenizing data...


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2345: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


### Creating Data Loaders

In [20]:
#from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
#train_labels = torch.tensor(y_train)
#test_labels = torch.tensor(y_test)

## For fine-tuning Bert, the authors recommmend a batch size of 16 or 32
#batch_size = 32

# Create the DataLoader for our training set
#train_data = TensorDataset(train_inputs,train_masks, train_labels)
#train_sampler = RandomSampler(train_data)
#train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
#test_data = TensorDataset(test_inputs, test_masks, test_labels)
#test_sampler = SequentialSampler(test_data)
#test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## Modeling

In [21]:
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassifier class

class BertClassifier(nn.Module):
    
    def __init__(self, freeze_bert=False):
        super(BertClassifier,self).__init__()
        
        # Specify hidden size of Bert, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50 , 6
        
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        self.classifier = nn.Sequential(
                            nn.Linear(D_in, H),
                            nn.ReLU(),
                            nn.Linear(H, D_out))
        
        # Freeze the Bert Model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
    
    def forward(self,input_ids,attention_mask):

        outputs = self.bert(input_ids=input_ids,
                           attention_mask = attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:,0,:]
        
        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)
        
        return logits

### An Initializer Function for Bert Classifier

In [22]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):

    # Define Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)
    
    bert_classifier.to(device)
    
    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),)
    
    # Total number of training steps
    total_steps = len(train_dataloader) * epochs
    
    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps=0, # Default
                                              num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [23]:
def set_seed(seed_value=42):

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

### Creating a Trainer Function for initialized Bert Model

In [24]:
import random
import time

# Specify loss function
loss_fn = nn.BCEWithLogitsLoss()

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            optimizer.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels.float())
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20--50000 batches
            if (step % 50000 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70) # printing for visualization
        
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print the time for training
            time_elapsed = time.time() - t0_epoch
            
            # printing for visualization
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


### Evaluation Function to calculate loss and accuracy of the classifier

In [25]:
def evaluate(model, val_dataloader):

    #Evaluation mode
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels.float())
        val_loss.append(loss.item())
        
        # Calculate the accuracy rate
        accuracy = accuracy_thresh(logits.view(-1,6),b_labels.view(-1,6))
        
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

def accuracy_thresh(y_pred, y_true, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy"
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()

### A Prediction method to predict test set with the classifier

In [26]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):

    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)
    
    #print(probs)
    # Apply softmax to calculate probabilities
    probs = all_logits.sigmoid().cpu().numpy()
    

    return probs

### Training on the whole data

In [27]:
# Concatenate the train set and the validation set
#all_train_data = torch.utils.data.ConcatDataset([train_data, test_data])
#all_train_sampler = RandomSampler(all_train_data)
#all_train_dataloader = DataLoader(all_train_data, sampler=all_train_sampler, batch_size=32 )

# Train the Bert Classifier on the entire training data
#set_seed(42)
#bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
#train(bert_classifier, all_train_dataloader, epochs=4)

In [28]:
#print('Tokenizing data...')
#test_inputs, test_masks = preprocessing_for_bert(test_data.content)

# Create the DataLoader for our test set
#test_dataset = TensorDataset(test_inputs, test_masks)

# No sampling to test on all test data
#test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=32)

In [29]:
# Get probabilites of predictions
#probs = bert_predict(bert_classifier, test_dataloader)

In [30]:
#submission = pd.DataFrame(probs,columns=['toxic', 'abusive', 'vulgar', 'menace', 'offense', 'bigotry'])
#test_data[['toxic', 'abusive', 'vulgar', 'menace', 'offense', 'bigotry']]=submission
#final_sub = test_data[['id','toxic', 'abusive', 'vulgar', 'menace', 'offense', 'bigotry']]
#final_sub.head()

In [31]:
#final_sub.to_csv('submissions.csv',index=False)
#final_sub.head()